# Introduction to operator overloading

Infix operators are the ones that are placed between two operands. 
Unary operator are the operators that receives only a input.
Parenthesis `()`, slicing `[]` and the dot `.` are also operators but will not be discussed here.

Things that cannot be done with operators in python:
- Change the meaning of operators to builtin types
- Create new operators, only overload existing ones
- Overload the following: `is`, `or`, `and`, `not`. Question: they are operators? They seem more like keywords. However, we can overload the binary operators `&`, `|` and `~`

General rule: always return a new object and do not modify the operands.

# Unary operators

The three main unary operators are:
- `-`, implemented by `__neg__`. If `x` is `2`, then `-x == 2`
- `+`, implemented by `__pos__`. In general `x == +x`, but when you want to modify this behavior, use this
- `+`, implemented by `__invert__`. Binary negation of a integer defined as `~x == -(x+1)`. If `x` is `2`, then `~x == -3`.

The `abs()` function could be considered a unary operator as well, and is implemented using `__abs__` dunder method.

We can implement those methods in `Vector` class:

```python
class Vector:
    # A lot of other defs

    def __abs__(self):
        return math.hypot(*self)
    
    def __neg__(self):
        return Vector(-x ofr x in self)

    def __pos__(self):
        Vector(self)
```


# Infix operators

Let's see the implementation of `+` and `*` operators for the Vector class

## Addition operator

We want it to sum the vectors component wise. If a vector is smaller than other, it should be filled with zeros and then summed against the bigger vector
```python
import itertools
class Vector:
    # other definitions

    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b, for a, b in pairs) # every pair is a tuple

v1 = Vector([3, 4, 5])
v1 + (10, 20, 30) # Vector([13.0, 24.0, 35.0])

v2d = Vector2d(1, 2)
v1 + v2d # Vector([4.0, 6.0, 5.0])
```

Because of duck typing, we can even sum a `Vector` and a `Vector2d` or a tuple, because `zip_longest` accepts any iterable. However, with the above version, it only provides this functionality for the right operand. If the order was switched for the examples above, we would receive a error. To fix this we just need to implement `__radd__` dunder method in the `Vector` class. See in the book the reasoning why it exists this function

```python
def __radd__(self, other):
    return self + other # delegates to `__add__`, because the addition is cumolativy in this case.

# When __radd__ just delegates to __add__ we can do
__radd__ = __add__
```

There still some problems with our implementation: if we pass a non-iterable object or a iterable that is composed of elements that cannot be summed to a `float`, we get obscure messages and wrong Error types. To fix this, only change the implementation to:

```python
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b, for a, b in pairs) # every pair is a tuple
        except TypeError:
            return NotImplemented
```